# 自定义组件v-model
- 一个组件上的v-model默认会利用名为value的prop属性和名为input的事件，但是像单选框，复选框等类型的输入控件可能会将value特性用于不同的目的，这时候可以在自定义组件的时候，通过设置model选项来实现不同的处理方式
- 示例代码如下：
    - 其中props定义的属性是给外面调用组件的时候使用的，model中定义的prop:"count"是告诉后面使用v-model的时候，要修改子组件中的哪个属性，event:"count-changed"是告诉v-model，后面触发哪个事件的时候要修改属性
              <div id='app'>
                <Stepper v-model="goods_count"></Stepper>

              </div>
              <script>
                // 计步器功能的实现
                Vue.component("Stepper", {
                  props: ['count'],
                  // model是用来配置v-model的表现形式,也就是说它是用来修改goods_count的
                  // 告诉组件什么时候该去更新goods_count这个值
                  model: {
                    // 告诉组件当这个事件被执行的时候修改goods_count这个值
                    // 又什么时候该执行这个事件呢？当执行-和+的时候，所以在methods中进行了定义
                    event: "count-changed",
                    // 你v-model传递进来的值我要绑定到子组件中的哪个值上面去
                    prop: "count",
                  },
                  template: `
                  <div>
                    <button @click="substract">-</button>
                    <span>{{count}}</span>
                    <button @click="add">+</button>
                  </div>
                  `,
                  methods:{
                    substract() {
                      // 当执行substract这个事件时，触发count-changed这个事件
                      this.$emit("count-changed", this.count-1);
                    },
                    add() {
                      this.$emit("count-changed", this.count+1);
                    },
                  }
                })
                new Vue({
                  el: '#app',
                  data: {
                    goods_count: 0,
                  }
                })
              </script>

# 插槽
- 定义完一个组件后，可能在使用的时候还需要向这个组件中插入新的元素或者文本，这时候就可以使用插槽来实现，通过在子组件中的模板中加入slot标签来实现。如果没有slot标签，那么在引用子组件时，该组件起始标签到结束标签之间的任何内容都会被抛弃

### 插槽作用域
- 默认在插槽中的代码只能使用全局Vue的属性，插槽读取的变量是从父组件中读取的，子组件中的变量它是读取不到的。
- 如果想要使用自定义组件中的属性，那么需要在定义slot的时候使用v-bind来进行绑定。
    - 把需要传递给插槽的变量使用v-bind绑定到slot上
    - 在使用插槽的时候，指定名称后，加上一个名称，这个名称可以随便取，那么绑定到slot上的所有属性都可以通过这个名称获取
    - 如果只有一个插槽，且未命名或者命名为default，那么就可以把这个属性定义放到组件上
            比如：
            <container v-slot="props">
                {{props.navs}}
            </container>
- 示例代码：
        <div id='app'>
            <container>
              <!-- 这个headerNav的名字可以随便取，主要是把里面的参数带出来 -->
              <template v-slot:header="headerNav">
                顶端内容：{{headerNav.navs}}
              </template>
              <template v-slot:footer="footerNav">
                底部内容：{{footerNav.address}}/{{footerNav.aboutus}}
              </template>
            </container>
          </div>
          <script>
            Vue.component("container", {
              template: `
              <div>
                <div class="header">
                  <slot name="header" v-bind:navs="navs"></slot>
                </div>
                <div class="footer">
                  <slot name="footer" v-bind:address="address" v-bind:aboutus="aboutus"></slot>
                </div>
              </div>
              `,
              data: function () {
                return{
                  address: "四川成都",
                  aboutus: "涛涛俱乐部",
                  navs: ['首页', '联系我们', '业务咨询']
                }
              }
            })
            new Vue({
              el: '#app',
              data: {

              }
            })
          </script>

### 插槽默认值
- 有时候在使用组件的时候，插槽中绝大部分情况是一种元素，那么就可以给他一个默认值，然后后面如果不想使用这个默认值的时候，就只需要提供自己定义的值就可以了
        <div id='app'>
            <nav-link :url="url">百度</nav-link>
            <button-confirm></button-confirm>
            <button-confirm>立即支付</button-confirm>
          </div>
          <script>
            Vue.component("nav-link", {
              props: ['url'],
              template: `
              <div>
                <a :href="url">
                  <slot></slot>  
                </a>
              </div>
              `,
            }),
            Vue.component("button-confirm", {
              template: `
              <button>
                <slot>确认</slot>
              </button>
              `,
            }),
            new Vue({
              el: '#app',
              data: {
                url: "https://www.baidu.com",
              }
            })
          </script>

## 命名插槽
- 自定义组件中可以有多个插槽，这时候就需要通过名字来进行区分，其实如果没有指定名字，默认是有一个名字叫做default的
- 具体使用：
    - 定义：<slot name="header"> </slot>
    - 使用：<template v-slot:header></template>
            <div id='app'>
            <container>
              <template v-slot:header>这是header</template>
              <template v-slot:body>嗯哼</template>
              <template v-slot:footer>这是footer</template>
              <!-- 没有命名的插槽就会放入没有命名的插槽中 -->
              默认的
            </container>
          </div>
          <script>
            Vue.component("container", {
              template: `
              <div>
                <div class="header">
                  <slot name="header"></slot>
                </div>
                <div class="body">
                  <slot name="body">这是body</slot>
                </div>
                <div class="footer">
                  <slot name="footer"></slot>
                </div>
                <div>
                  <slot></slot>  
                </div>
              </div>
              `,
            })
            new Vue({
              el: '#app',
              data: {

              }
            })
          </script>

